In [1]:
!pip install gtts


In [2]:
!pip install googletrans==4.0.0-rc1

In [3]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from gtts import gTTS
import os
import speech_recognition as sr 
import nltk
from nltk.stem import WordNetLemmatizer
from googletrans import Translator

In [4]:
# Download NLTK resources
nltk.download('popular', quiet=True)
nltk.download('nps_chat', quiet=True)
nltk.download('punkt') 
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naimu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naimu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Load chat posts
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

In [6]:
# Function to extract dialogue act features
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

In [7]:
# Prepare featuresets
featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [8]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

In [9]:
# Function to greet user
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [10]:
# Function to normalize text
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [11]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)


In [12]:
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [13]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Check if the file exists in the current directory
file_path = os.path.join(current_directory, 'intro_join')
if os.path.exists(file_path):
    print("File 'intro_join' exists in the directory:", current_directory)
else:
    print("File 'intro_join' does not exist in the directory:", current_directory)


File 'intro_join' does not exist in the directory: C:\Users\naimu\Music\Speech Recognition ML DL AI


In [14]:
# Read input corpus
file_path = "C:\\Users\\naimu\\Music\\Speech Recognition ML DL AI\\intro_join.txt"
with open(file_path, 'r', encoding='utf8', errors='ignore') as fin:
    raw = fin.read().lower()

In [15]:
# Tokenization
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [16]:
# Preprocessing
lemmer = WordNetLemmatizer()

In [17]:
# Function to generate response
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if req_tfidf == 0:
        robo_response = "I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response + sent_tokens[idx]
        return robo_response


In [18]:
# Recording voice input using microphone 
file = "file.mp3"
flag = True
fst = "My name is Jarvis. I will answer your queries about Science. If you want to exit, say Bye"
os.system("echo {} | text2wave -o {}.wav".format(fst, file))

r = sr.Recognizer()

In [19]:
print(fst)

My name is Jarvis. I will answer your queries about Science. If you want to exit, say Bye


In [20]:
# Google translator instance
translator = Translator()


In [ ]:
google_translate_link = "https://translate.google.com/"

# Taking voice input and processing 
while flag:
    with sr.Microphone() as source:
        audio = r.listen(source)
    try:
        # Recognize speech using Google Speech Recognition
        user_response = r.recognize_google(audio)
        print("YOU SAID:", user_response)
        
        # Detect language
        detected_language = translator.detect(user_response).lang
        print("Detected language:", detected_language)
        
        # Translate to English
        if detected_language != 'en':
            user_response = translator.translate(user_response, src=detected_language, dest='en').text
            print("Translated to English:", user_response)
    except sr.UnknownValueError:
        print("Oops! Didn't catch that")
        continue
    
    clas = classifier.classify(dialogue_act_features(user_response))
    if clas != 'Bye':
        if clas == 'Emotion':
            flag = False
            print("Jarvis: You are welcome..")
        else:
            if greeting(user_response) is not None:
                print("Jarvis:", greeting(user_response))
            else:
                print("Jarvis:", end="")
                res = response(user_response)
                print(res)
                sent_tokens.remove(user_response)
                tts = gTTS(res, 'en')
                tts.save(file)
                os.system("mpg123 " + file)
    else:
        flag = False
        print("Jarvis: Bye! Take care..")

print("You can access Google Translate here:", google_translate_link)


YOU SAID: hello
Detected language: en
Jarvis: *nods*


In [ ]:

# Taking voice input and processing 
while flag:
    with sr.Microphone() as source:
        audio = r.listen(source)
    try:
        # Recognize speech using Google Speech Recognition
        user_response = r.recognize_google(audio)
        print("YOU SAID:", user_response)
        
        # Detect language
        detected_language = translator.detect(user_response).lang
        print("Detected language:", detected_language)
        
        # Translate to English
        if detected_language != 'en':
            user_response = translator.translate(user_response, src=detected_language, dest='en').text
            print("Translated to English:", user_response)
    except sr.UnknownValueError:
        print("Oops! Didn't catch that")
        continue
    
    clas = classifier.classify(dialogue_act_features(user_response))
    if clas != 'Bye':
        if clas == 'Emotion':
            flag = False
            print("Jarvis: You are welcome..")
        else:
            if greeting(user_response) is not None:
                print("Jarvis:", greeting(user_response))
            else:
                print("Jarvis:", end="")
                res = response(user_response)
                print(res)
                sent_tokens.remove(user_response)
                tts = gTTS(res, 'en')
                tts.save(file)
                os.system("mpg123 " + file)
    else:
        flag = False
        print("Jarvis: Bye! Take care..")
